# Download dependencies/data

In [1]:
!curl -L https://www.dropbox.com/s/rndzndlgpw3coiq/kaggle-download.zip -o kaggle-download.zip
!unzip kaggle-download.zip -d kaggle_data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17    0    17    0     0     57      0 --:--:-- --:--:-- --:--:--    57
100   340  100   340    0     0    478      0 --:--:-- --:--:-- --:--:--     0
100   534    0   534    0     0    425      0 --:--:--  0:00:01 --:--:--   425
 41  515M   41  212M    0     0  46.1M      0  0:00:11  0:00:04  0:00:07 70.7M^C
Archive:  kaggle-download.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of kaggle-download.zip or
        kaggle-download.zip.zip, and cannot find kaggle-download.zip.ZIP, period.


In [ ]:
pip install xgboost

In [109]:
pip install hyperopt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.6 MB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 840 kB 57.8 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 53.3 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492026 sha256=ec372d44630e94bb8052c2a3a28630c7890d98d786025dd7107cdf093e6684fb
  Stored in directory: /home/acz001/.cache/pip/wheels/bf/5d/6a/2e53874f7ec4e2bede522385439531fafec8fafe005b5c3d1b
Successfully built future
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import xgboost
import sklearn

# Processing data

In [2]:
df_taxi_data = pd.read_csv("kaggle_data/metaData_taxistandsID_name_GPSlocation.csv")
# The following code normalizes longitudes and latitudes.
# df_taxi_data["Longitude"] = (df_taxi_data["Longitude"] - df_taxi_data["Longitude"].mean()) / df_taxi_data["Longitude"].std()
# df_taxi_data["Latitude"] = (df_taxi_data["Latitude"] - df_taxi_data["Latitude"].mean()) / df_taxi_data["Latitude"].std()

In [3]:
from datetime import datetime
import math

def process_df(df):
    def parse_time(x):
        # We are using python's builtin datetime library
        # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

        # Each x is essentially a 1 row, 1 column pandas Series
        dt = datetime.fromtimestamp(x["TIMESTAMP"])
        return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

    def get_lat_long(x):
        if math.isnan(x["ORIGIN_STAND"]):
            return 0, 0
        taxi_row = df_taxi_data.iloc[int(x["ORIGIN_STAND"]) - 1]
        return float(taxi_row["Latitude"]), float(taxi_row["Longitude"])
    
    def augment_lat_long(df):
        if "POLYLINE" not in df:
            return df
        augmented_df = []
        for i, row in df.iterrows():
            if math.isnan(row["ORIGIN_STAND"]):
                    start = [i.replace(']', '').replace('[', '').split(',') for i in row["POLYLINE"].split('],')][0]
                    if len(start) == 2:
                        row.LATITUDE = float(start[0])
                        row.LONGITUDE = float(start[1])
                        augmented_df.append(row)
        return pd.concat([df, pd.DataFrame(augmented_df, columns=df.columns)], ignore_index=True)

    df[["LATITUDE", "LONGITUDE"]] = df.apply(get_lat_long, axis=1, result_type="expand")
    df[["YR", "MON", "DAY", "HR", "WK"]] = df.apply(parse_time, axis=1, result_type="expand")
    #df = augment_lat_long(df)
    df.drop(['TRIP_ID', "POLYLINE", 'MISSING_DATA', 'TAXI_ID', 'DAY_TYPE'], axis=1, inplace=True, errors="ignore")
    
    categorical_cols = ['CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'YR', 'MON', 'DAY', 'HR', 'WK']
    for col in categorical_cols:
        df[col] = df[col].astype('category')
    if 'LEN' in df.columns:
        df['LEN'] = df['LEN'].astype('float64')
    
    return df

In [4]:
# These are all of the files you are given
original_df_tr = pd.read_csv("kaggle_data/train.csv")

# getting rid of incomplete rows
original_rows = original_df_tr.shape[0]

df_tr = original_df_tr.loc[original_df_tr['MISSING_DATA'] == False]

rows_removed = original_rows - df_tr.shape[0]
print(f"{rows_removed=}")

rows_removed=10


In [5]:
def polyline_to_trip_duration(polyline):
    return max(polyline.count("[") - 2, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)

df_tr = process_df(df_tr)

/tmp/ipykernel_5873/3024709668.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)
/tmp/ipykernel_5873/2434493553.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["LATITUDE", "LONGITUDE"]] = df.apply(get_lat_long, axis=1, result_type="expand")
/tmp/ipykernel_5873/2434493553.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [6]:
len_mean = df_tr['LEN'].mean()
len_std = df_tr['LEN'].std()
len_range = (len_mean - 3 * len_std, len_mean + 3 * len_std)
df_tr = df_tr.loc[df_tr['LEN'] < len_mean + 3 * len_std]

In [7]:
from sklearn.model_selection import train_test_split

X = df_tr.drop('LEN', axis=1)
y = df_tr['LEN']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.1)

# Model training

In [8]:
import xgboost as xgb

In [9]:
# These are all of the files you are given
# Writing the output to submit to kaggle
df_test = pd.read_csv("kaggle_data/test_public.csv")
titles = df_test['TRIP_ID']
df_test = process_df(df_test)

In [10]:
model = xgb.Booster()
model.load_model('model.json')
X_submission = df_test
dsubmission_reg = xgb.DMatrix(X_submission, enable_categorical=True)

In [11]:
predictions = model.predict(dsubmission_reg)
df = pd.DataFrame(data={"TRIP_ID": titles, "TRAVEL_TIME": predictions})
df.to_csv("solution.csv", index=False)